# Decoradores

## Tabla de contenidos
***
- [Decorando funciones](#id1)
<a href='#id1'></a>

- [La importancia de functools.wraps](#id2)
<a href='#id2'></a>

- [Concatenando o anidando decoradores](#id3)
<a href='#id3'></a>




***

Los decoradores son en esencia wrappers de funciones/classes que pueden ser utilizados para modificar el input, output, o incluso la clase/función en si misma antes de ejecutarla.

Esto puede ser logrado teniendo otra función que llama a la función interior, o mediante herencia de ciertas classes llamadas **mixins**.

Los decoradores brindan mucho "poder de reutilizar". Algunos decoradores muy útiles que ya vienen implementados en Python son 
`@property`, `@classmethod`, `@staticmethod`.

Es importante hacer notar que "envolver" una función crea una nueva función y lo hace más difícil alcanzar el interior de la función y sus propiedades. Un ejemplo de esto es la funcionalidad `help(funcion)` que ya viene implementada en Python.

<a id='id1'></a>

## Decorando funciones

Los decoradores son funciones o classes que "envuelven" otras funciones y/o classes.

In [9]:
def decorador(funcion):
    print(f"He ejecutado la función {funcion.__name__}")
    return funcion

def suma(a, b):
    return a + b

suma = decorador(suma)

He ejecutado la función suma


Para hacer la syntaxís más fácil de leer, Python permite decorar una función usando el operador `@` como atajo.

In [11]:
@decorador
def sumas(a,b):
    return a + b

He ejecutado la función sumas


Algunos usos de los decoradores incluye
- Registrar una función/clase
- Modificar el input de una función/clase
- Modificar el output de una función/clase
- Loggear instancias de clases o llamados de funciones

In [12]:
import functools

def decorador(funcion):
    #Este decorador asegura que imitamos la función que "envolvemos"
    @functools.wraps(funcion)
    def _decorador(a , b):
        #Pasa los argumentos modificados a la función
        result = funcion(a, b + 5)
        
        #Printeamos el nombre
        nombre = funcion.__name__
        print(f"{nombre}(a = {a}, b = {b}): {result}")
        
        #Retornamos un resultado modificado
        return result + 4
    
    return _decorador


@decorador
def funcion(a,b):
    return a + b

In [13]:
funcion(1,2)

funcion(a = 1, b = 2): 8


12

Como se puede observar, podemos modificar, añadir y/o remover argumentos. Podemos modificar el valor que retornamos o incluso llamar a una función completamente distinta si así lo deseamos. Además, de que podemos printear en consola todo el comportamiento si es necesario, lo cual puede ser bastante útil para debuggear.

## Decoradores de funciones genéricas

Si queremos hacer el generador más genérico, podemos reemplazar a, b con `*args` y `**kwargs**` para obtener los *arguments* y *keyword arguments*, respectivamente. Sin embargo, esto introduce un nuevo problema. O necesitamos asegurarnos de solo usar argumentos regulares o *keyword arguments* o el chequeo se puede volver increíblemente difícil.

In [2]:
import functools

def decorator(function):
    @functools.wraps(function)
    def _decorator(*args, **kwargs):
        a, b = args
        return function(a, b + 5)
    
    return _decorator

@decorator
def func(a, b):
    return a + b

In [3]:
func(1, 2)

8

In [4]:
func(a = 1, b = 2)

ValueError: not enough values to unpack (expected 2, got 0)

Como se puede observar, en este caso los *keyword arguments* están rotos. Para trabajar alrededor de esto, tenemos distintos métodos. Podemos cambiar los argumentos para argumentos únicamente posicionales o *keyword-only arguments*.

## La importancia de functools.wraps

Cuando sea que estés creando un decorador, siempre asegúrate de añadir `functools.wraps` para "envolver" la función interior. Sin "envolverla", vas a perder todas las propiedades originales de la función, lo que puede ocasionar un comportamiento confuso e inesperado.

In [14]:
import functools

def decorador(funcion):
    @functools.wraps(funcion)
    def _decorador(*args, **kwargs):
        return funcion(*args, **kwargs)
    
    return _decorador



@decorador
def suma(a, b):
    """Se suman a y b"""
    return a + b

In [15]:
help(suma)

Help on function suma in module __main__:

suma(a, b)
    Se suman a y b



In [17]:
suma.__name__

'suma'

El funcionamiento de `functools.wraps` es para nada mágico; copia y actualiza muchos atributos. Específicamente, los siguientes atributos son copiados:

- `__doc__`
- `__name__`
- `__module__`
- `__annotations__`
- `__qualname__`

<a id='id3'></a>

## Concatenando o anidando decoradores

Dado que estamos "envolviendo" funciones, no hay nada que nos impida el añadir múltiples decoradores. Los decoradores son inicializados partiendo desde adentro, pero son llamados partiendo por afuera.

In [18]:
import functools

def track(funcion = None, label = None):
    #Truco para añadir un argumento opcional a nuestro decorador
    if label and not funcion:
        return functools.partial(track, label = label)
    
    print(f"Inicializando {label}")
    
    @functools.wraps(funcion)
    def _track(*args, **kwargs):
        print(f"Llamando {label}")
        funcion(*args, **kwargs)
        print(f"Se llamó {label}")
    
    return _track

In [19]:
@track(label = 'exterior')
@track(label = "interior")
def funcion():
    print("Se generó un llamado de la función")

Inicializando interior
Inicializando exterior


In [20]:
funcion()

Llamando exterior
Llamando interior
Se generó un llamado de la función
Se llamó interior
Se llamó exterior


Como se puede observar los decoradores son llamados desde el exterior al interior antes de correr la función y al correrla son llamados desde el interior al exterior cuando se procesan los resultados.

## Registrando funciones usando decoradores

A continuación veremos como usar decoradores para registrar una función que puede ser útil para registrar plugins, callbacks y más. Esto puede ser muy útil en una interfaz de usuario (GUI). Creando un sistema que puede registrar callbacks, podemos hacer que un botón emita una señal de cliqueado y conecte funciones a ese evento.

In [5]:
import collections

class EventRegistry:
    def __init__(self):
        self.registry = collections.defaultdict(list)
    
    
    def on(self, *events):
        def _on(function):
            for event in events:
                self.registry[event].append(function)
            
            return function
        
        return _on
    
    
    def fire(self, event, *args, **kwargs):
        for function in self.registry[event]:
            function(*args, **kwargs)

In [6]:
events = EventRegistry()

In [7]:
@events.on('success', 'error')
def teardown(value):
    print(f'Tearing down got: {value}')

@events.on('success')
def success(value):
    print(f'Successfully executed: {value}')

In [8]:
events.fire('success', 'Everything is fine')

Tearing down got: Everything is fine
Successfully executed: Everything is fine


In [30]:
events.fire('error', 'Oops, some error here')

Tearing down got: Oops, some error here
Tearing down got: Oops, some error here


In [9]:
events.fire('non-existing', 'nothing to see here')

Si bien este ejemplo es bastante básico, este patrón puede ser aplicado en muchos escenarios: manejo de eventos para un web server, permitir que plugins se registren a si mismos, permitir que plugins se registren en una aplicación, entre otros.

## Memorización usando decoradores

La memorización es un truco para recordar los resultados y así hacer correr el código de forma mucho más rápida en escenarios específicos. El truco es guardar un mapeo del input y del output esperado, así solo se tiene que calcular el valor una única vez. Un ejemplo de esto son los números de Fibonacci.

In [1]:
import functools

def memoize(function):
    #Se guarda el caché como un atributo de la función
    #Así podemos aplicar el decorador a múltiples funciones sin
    #compartir el caché
    
    function.cache = dict()
    
    @functools.wraps(function)
    def _memoize(*args):
        #Si el caché no está disponible, se llamara a la función
        #Notar que todos los argumentos deben ser hasheables
        
        if args not in function.cache:
            function.cache[args] = function(*args)
            
        return function.cache[args]
    
    
    return _memoize

In [3]:
@memoize
def fibonacci(n):
    if n < 2:
        return n
    
    else:
        return fibonacci(n - 1) + fibonacci(n - 2) 
    
for i in range(1, 7):
    print(f"fibonacci {i}, {fibonacci(i)}")

fibonacci 1, 1
fibonacci 2, 1
fibonacci 3, 2
fibonacci 4, 3
fibonacci 5, 5
fibonacci 6, 8


In [4]:
fibonacci.__wrapped__.cache

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8}

Implementar la función memoize por su propia cuenta es generalmente no muy útil, dado que Python introdujo `lru_cache`. Lo que hace es similar a la función que acabamos de implementar, pero es más avanzado. Mantiene un tamaño fijo de caché para guardar memoria, y guarda estadísticas así se puede chequear si el tamaño debe ser extendido.

Si solo gusta observar las estadísticas y no hay necesidad de guardar cosas en el caché, se puede setear el `maxsize` a 0. Con un tamaño fijo, `lru_cache` va a mantener solo los elementos a los cuales se ha accedido recientemente y va a desechar los más antiguos una vez se llena.En la mayoría de los casos se sugiere usar `lru_cache` en vez de el propio decorador.

In [10]:
import functools

#Se crea un decorador que cuenta las veces que es llamado
def counter(function):
    function.calls = 0
    @functools.wraps(function)
    def _counter(*args, **kwargs):
        function.calls += 1
        return function(*args, **kwargs)
    
    return _counter

#Se crea un caché LRU de tamaño 3
@functools.lru_cache(maxsize=3)
@counter
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [11]:
fibonacci(100)

354224848179261915075

In [12]:
fibonacci.cache_info()

CacheInfo(hits=98, misses=101, maxsize=3, currsize=3)

In [13]:
fibonacci.__wrapped__.__wrapped__.calls

101

Aquí el orden de ejecución es
- functools.lru_cache
- counter
- fibonacci
y el orden de retorno de valores es en el sentido contrario.

## Decoradores con argumentos (opcionales)

Los decoradores pueden aceptar argumentos, pues también son funciones, pero esto añade una "capa" extra al decorador. 

La única advertencia es que el argumento opcional no sea calleable. Esto significa que necesitamos chequear los argumentos del decorador para ver si son el método decorado o un argumento regular. Si el argumento es calleable, se necesitará pasarlo como una keyword.

In [15]:
import functools

def add(function = None, add_n = 0):
    #Si la función no es calleable, es probablemente add_n
    if not callable(function):
        #Testeamos de que no pasamos None como add_n
        
        if function is not None:
            add_n = function
        
        return functools.partial(add, add_n = add_n)
    
    @functools.wraps(function)
    def _add(n):
        return function(n) + add_n
    
    return _add

In [17]:
@add
def add_zero(n):
    return n

add_zero(5)

5

In [20]:
@add(1)
def add_one(n):
    return n

add_one(5)

6

Este decorador usa `callable()` para testear si el argumento es calleable, como una función.

## Creando decoradores usando classes

De forma similar como creamos decoradores como funciones, podemos crear decoradores usando classes. Esto lo hace más conveniente para guardar datos, herencia y reusar de forma mucho más conveniente que con funciones.

In [21]:
import functools

class Debug(object):
    def __init__(self, function):
        self.function = function
        #functools.wraps para classes
        functools.update_wrapper(self, function)
        
    
    def __call__(self, *args, **kwargs):
        output = self.function(*args, **kwargs)
        name = self.function.__name__
        print(f"{name}({args!r}, {kwargs!r}): {output!r}")
        return output

In [25]:
@Debug
def add(a, b =0):
    return a + b

output = add(a=4, b=2)

add((), {'a': 4, 'b': 2}): 6


La única diferencia entre funciones y classes es que `functools.wraps` es reemplazado por `functools.update_wrapper` en el método `__init__`

## Decorando funciones de classes

Decorar funciones de classes es similar a funciones regulares, pero se debe ser consciente del primer argumento, `self` - la instancia de clase. Algunos de los decoradores más usados en classes son `classmethod`, `staticmethod` y `property`.

## Saltándose la instancia - classmethod y staticmethod
La principal diferencia entre `classmethod` y `staticmethod`, es que `classmethod` pasa un objeto de clase en vez de una instancia de clase (self), y `staticmethod` se salta ambos, la clase y la instancia completamente.

Nota: `locals()` en Python es un *built-in* que muestra todas las variables locales. De forma similar, `globals()` es una función que también está disponible.

In [29]:
import pprint

class Spam(object):
    def some_instancemethod(self, *args, **kwargs):
        pprint.pprint(locals(), width=60)
        
    
    @classmethod
    def some_classmethod(cls, *args, **kwargs):
        pprint.pprint(locals(), width=60)
        
    
    @staticmethod
    def some_staticmethod(*args, **kwargs):
        pprint.pprint(locals(), width=60)

In [31]:
spam = Spam()

In [32]:
spam.some_instancemethod(1,2, a=3, b=4)

{'args': (1, 2),
 'kwargs': {'a': 3, 'b': 4},
 'self': <__main__.Spam object at 0x00000171AA8CD220>}


In [34]:
Spam.some_instancemethod()

TypeError: some_instancemethod() missing 1 required positional argument: 'self'

In [35]:
#Si añadimos parametros, nuestro primer argumento es usado como argumento
#Esto puede ocasionar errores extrañox e inesperados
Spam.some_instancemethod(1,2, a=3, b=4)

{'args': (2,), 'kwargs': {'a': 3, 'b': 4}, 'self': 1}


In [36]:
spam.some_classmethod(1,2, a=3, b=4)

{'args': (1, 2),
 'cls': <class '__main__.Spam'>,
 'kwargs': {'a': 3, 'b': 4}}


In [37]:
Spam.some_classmethod(1,2, a=3, b=4)

{'args': (1, 2),
 'cls': <class '__main__.Spam'>,
 'kwargs': {'a': 3, 'b': 4}}


La principal diferencia es que aquí, en vez de `self`, tenemos `cls`, el cual contiene la class Spam en vez de la instancia spam.

In [38]:
spam.some_staticmethod(1, 2, a=3, b=4)

{'args': (1, 2), 'kwargs': {'a': 3, 'b': 4}}


In [40]:
Spam.some_staticmethod()

{'args': (), 'kwargs': {}}


In [41]:
Spam.some_staticmethod(1, 2, a=3, b=4)

{'args': (1, 2), 'kwargs': {'a': 3, 'b': 4}}


"Descriptors" pueden ser usados para modificar el comportamiento de los atributos. Esto significa que si un descriptor es usado como el valor de un atributo, se puede modificar que valor está siendo seteado, obtenido, o deleteado cuando estas operaciones son llamadas en el atributo.

In [43]:
class Spam:
    def __init__(self, spam=1):
        self.spam = spam
        
    
    def __get__(self, instance, cls):
        return self.spam + instance.eggs
    
    
    def __set__(self, instance, value):
        instance.eggs = value - self.spam
        
        
class Sandwich:
    spam = Spam(5)
    
    def __init__(self, eggs):
        self.eggs = eggs

In [44]:
sandwich = Sandwich(1)

In [45]:
sandwich.eggs

1

In [46]:
sandwich.spam

6

In [47]:
sandwich.eggs = 10
sandwich.spam

15

Cuando seteamos u obtenemos valores de `sandwich.spam`, llama a `__get__` o `__set__` en Spam.

## Properties - uso inteligente de descriptors

El decorador `property` es probablemente uno de los decoradores más utilizados en Python. Permite añadir getters/setters a propiedades existentes de una instancia, permitiendo que se puedan añadir validadores y modificar los valores antes de setearlos en las propiedades de la instancia.

Este decorador, puede ser usado como un asignador y decorador.

In [5]:
import functools

class Precios:
    
    def __init__(self, precio):
        self._precio = precio
        self._precio_maximo = 100
    
    @property
    def precio(self):
        return self._precio
    
    @precio.setter
    def precio(self, nuevo_precio):
        
        if nuevo_precio < self._precio_maximo:
            print(f"El valor ha sido modificado a {nuevo_precio}")
            self._precio = nuevo_precio
            
        else:
            print(f"Usted no puede asignar {nuevo_precio}, ya que supera el precio máximo")
            
    @precio.deleter
    def precio(self):
        print(f"Se ha eliminado el atributo self._precio")
        del self._precio

In [6]:
precio = Precios(5)

In [7]:
precio.precio = 3

El valor ha sido modificado a 3


In [8]:
precio.precio = 200

Usted no puede asignar 200, ya que supera el precio máximo


In [9]:
del precio.precio

Se ha eliminado el atributo self._precio


## Decorando classes

Los decoradores de classes no son muy distintos de los regulares, excepto por el hecho de que toman una clase en vez de una función. Como es en el caso de funciones, esto ocurre cuando se declaran las clases y **no** cuando se instancian/llaman.

## Singletons - Classes con una sola instancia
Los Singletons son classes que permiten que exista solo una instancia. Así, en vez de obtener una classe específica para cierto llamado, siempre se obtiene la misma

## Dataclasses
Estas nos permiten definir objetos ordinarios con una syntaxis para especificar atributos. La función `dataclass` es aplicada como un decorador de clase. Por default, incluyen un comparador de igualdad, si todos los atributos son iguales, los objetos son iguales. Por default, las dataclasses no soportan comparaciones. El parámetro `order = True` permite la creación de esos métodos. Las comparaciones se realizarán en el orden en el que son definidos los atributos.

In [46]:
from dataclasses import dataclass

@dataclass(order = True)
class Persona:
    nombre: str
    apellido: str
    edad: int
        
    def reconocimiento(self):
        return (f"Mi nombre completo es {self.nombre} {self.apellido} y tengo {self.edad} años")

In [47]:
Kai = Persona(nombre="Kai", apellido="Yamamoto", edad=20)

In [48]:
Kai.reconocimiento()

'Mi nombre completo es Kai Yamamoto y tengo 20 años'

In [49]:
Francisco = Persona(nombre = "Francisco", apellido = "Solis", edad = 20)

In [50]:
Francisco.reconocimiento()

'Mi nombre completo es Francisco Solis y tengo 20 años'

In [51]:
Kai == Francisco

False

In [52]:
Kai > Francisco

True

In [57]:
help(dataclass)

Help on function dataclass in module dataclasses:

dataclass(cls=None, /, *, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
    Returns the same class as was passed in, with dunder methods
    added based on the fields defined in the class.
    
    Examines PEP 526 __annotations__ to determine fields.
    
    If init is true, an __init__() method is added to the class. If
    repr is true, a __repr__() method is added. If order is true, rich
    comparison dunder methods are added. If unsafe_hash is true, a
    __hash__() method function is added. If frozen is true, fields may
    not be assigned to after instance creation.



## contextmanager - with statements de forma fácil

Usar la classe `contextmanager`, podemos crear "context wrappers" def forma sencilla. Los "context wrappers" son usados cuando sea que uses un `with` statement. Un ejemplo de esto es la función `open`.

La forma estándar para crear un "context manager" es creando una clase que implementa los métodos `__enter__` y `__exit__`.

In [58]:
class Open:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        
    
    def __enter__(self):
        self.handle = open(self.filename, self.mode)
        return self.handle
    
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.handle.close()

In [59]:
with Open('test.txt', 'w') as fh:
    print("Nuestro testeo fue completado", file=fh)

Si bien funciona perfectamente, con `contextlib.contextmanager`, podemos lograr el mismo comportamiento en una menor cantidad de líneas.

In [60]:
import contextlib

@contextlib.contextmanager
def open_context_manager(filename, mode='r'):
    fh = open(filename, mode)
    yield fh
    fh.close()
    
with open_context_manager('test.txt', 'w') as fh:
    print("El testeo con la segunda clase fue completado", file=fh)

**Con archivos, conecciones a bases de datos y conexiones, es importante el siempre tener `close()` para así limpiar/liberar los recursos**